importing all the necessary libs for this 

In [47]:
import pandas as pd
import numpy as np
from pathlib import Path

In [48]:
project_root = Path.cwd().parent  # takes me to where the Readme file is 
raw_files = Path(project_root/"data/raw")
cleaned = Path(project_root/"data/cleaned")
cleaned.mkdir(exist_ok=True)
files = list(raw_files.glob("*.csv"))


In [49]:
for f in files:
    df = pd.read_csv(f)
    df = df.drop([0,1]).reset_index(drop=True)
    
    df = df.rename(columns={"Price":"Date"})

    out_file = cleaned/ f.name
    df.to_csv(out_file, index=False)
    
    print(f"Cleaned {f.name}")
    display(df.head(2))

    # for some reason its not dropping the 1st row 

Cleaned AXISBANK.NS_prices.csv


,Date,Close,High,Low,Open,Volume
0,2025-03-19,1051.1517333984375,1057.646183234451,1042.6089852123262,1046.9053623518132,4822365
1,2025-03-20,1053.44970703125,1057.9958707491392,1046.0061170837591,1055.3481090881514,3742644


Cleaned BHARTIARTL.NS_prices.csv


,Date,Close,High,Low,Open,Volume
0,2025-03-19,1623.5770263671875,1628.4364265129727,1614.0070066934181,1621.444827000761,6283464
1,2025-03-20,1691.1124267578125,1695.4759722582721,1632.4527924932477,1646.2375771399904,12074465


Cleaned HDFCBANK.NS_prices.csv


...

,Date,Close,High,Low,Open,Volume
0,2025-03-19,1242.1888427734375,1248.2645525643281,1233.8720835165427,1236.113132982547,16162399
1,2025-03-20,1264.101318359375,1267.9359786597456,1245.0773228865291,1246.8700894970418,15971477


Cleaned SBIN.NS_prices.csv


,Date,Close,High,Low,Open,Volume
0,2025-03-19,730.4331665039062,734.5505041985215,724.551272606947,725.0414304316936,7604850
1,2025-03-20,734.7955932617188,737.8836115337692,729.5509283979652,733.2761159507572,6963197


Cleaned TCS.NS_prices.csv


,Date,Close,High,Low,Open,Volume
0,2025-03-19,3454.56689453125,3503.8598932625678,3420.634577218764,3503.8598932625678,3051244
1,2025-03-20,3519.813232421875,3535.4211137803227,3469.285438744659,3469.285438744659,2845192


Preprocessinf the data to get returns and Volitility


Date      object
Close     object
High      object
Low       object
Open      object
Volume    object
dtype: object

In [55]:
processed = Path(project_root / "data/processed")
processed.mkdir(exist_ok=True)

cleaned_files = list(cleaned.glob("*.csv"))


for f in cleaned_files:
    df = pd.read_csv(f,parse_dates=['Date'])  # this will make the date col from object to datetime, probs
    df = df.sort_values("Date")
    

    # daily return
    df["daily_return"] = df['Close'].pct_change() # calculates fractional change = Current-Previous/Previous

    #  Rolling volatility (20-day window, annualized)
    df["vol_20"] = df["daily_return"].rolling(20).std() * np.sqrt(252)

    #  Mock sentiment (short-term momentum)
    df["sentiment"] = df["daily_return"].rolling(5).mean().fillna(0)
    df["sentiment"] = df["sentiment"].clip(-1, 1)

    # Save processed file
    out_file = processed / f.name.replace(".csv", "_processed.csv")
    df.to_csv(out_file, index=False)

    print(f"Processed {f.name}")
    display(df.head())


    



Processed AXISBANK.NS_prices.csv


,Date,Close,High,Low,Open,Volume,daily_return,vol_20,sentiment
0,2025-03-19,1051.151733,1057.646183,1042.608985,1046.905362,4822365,NaN,NaN,0.0
1,2025-03-20,1053.449707,1057.995871,1046.006117,1055.348109,3742644,0.002186,NaN,0.0
2,2025-03-21,1069.236206,1074.631619,1049.652967,1053.449649,6848565,0.014986,NaN,0.0
3,2025-03-24,1096.912476,1104.006388,1072.783074,1081.076035,8284863,0.025884,NaN,0.0
4,2025-03-25,1117.594971,1122.540695,1095.413853,1109.002216,11720528,0.018855,NaN,0.0


Processed BHARTIARTL.NS_prices.csv


,Date,Close,High,Low,Open,Volume,daily_return,vol_20,sentiment
0,2025-03-19,1623.577026,1628.436427,1614.007007,1621.444827,6283464,NaN,NaN,0.0
1,2025-03-20,1691.112427,1695.475972,1632.452792,1646.237577,12074465,0.041597,NaN,0.0
2,2025-03-21,1711.492188,1717.045736,1677.972360,1695.476056,14715349,0.012051,NaN,0.0
3,2025-03-24,1704.550171,1725.574362,1700.632846,1725.574362,4120999,-0.004056,NaN,0.0
4,2025-03-25,1716.153076,1726.070170,1691.955343,1710.698675,4857399,0.006807,NaN,0.0


Processed HDFCBANK.NS_prices.csv


...

,Date,Close,High,Low,Open,Volume,daily_return,vol_20,sentiment
0,2025-03-19,1242.188843,1248.264553,1233.872084,1236.113133,16162399,NaN,NaN,0.0
1,2025-03-20,1264.101318,1267.935979,1245.077323,1246.870089,15971477,0.017640,NaN,0.0
2,2025-03-21,1271.272583,1275.904109,1265.047446,1269.927978,21780769,0.005673,NaN,0.0
3,2025-03-24,1296.920166,1300.804676,1279.141198,1285.864347,20299319,0.020175,NaN,0.0
4,2025-03-25,1280.336426,1302.497915,1277.497787,1302.497915,13713899,-0.012787,NaN,0.0


Processed SBIN.NS_prices.csv


,Date,Close,High,Low,Open,Volume,daily_return,vol_20,sentiment
0,2025-03-19,730.433167,734.550504,724.551273,725.041430,7604850,NaN,NaN,0.0
1,2025-03-20,734.795593,737.883612,729.550928,733.276116,6963197,0.005972,NaN,0.0
2,2025-03-21,738.373779,740.726501,732.295810,735.187753,9577327,0.004870,NaN,0.0
3,2025-03-24,765.430481,767.538184,740.236356,743.079295,12095064,0.036644,NaN,0.0
4,2025-03-25,757.636963,770.528138,754.058846,768.469499,15190143,-0.010182,NaN,0.0


Processed TCS.NS_prices.csv


,Date,Close,High,Low,Open,Volume,daily_return,vol_20,sentiment
0,2025-03-19,3454.566895,3503.859893,3420.634577,3503.859893,3051244,NaN,NaN,0.0
1,2025-03-20,3519.813232,3535.421114,3469.285439,3469.285439,2845192,0.018887,NaN,0.0
2,2025-03-21,3534.581543,3572.267448,3422.857022,3432.735398,4161925,0.004196,NaN,0.0
3,2025-03-24,3584.812988,3605.360057,3523.221462,3556.215140,1834751,0.014211,NaN,0.0
4,2025-03-25,3613.262939,3664.877453,3593.753147,3600.668010,3135390,0.007936,NaN,0.0
